In [385]:
# Чтобы скачать и распаковать zip-архив в output папку в Питоне:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go 

from IPython.display import display

#!/usr/bin/env python
import io
import zipfile
import requests  # $ pip install requests

pd.set_option('display.max_columns', 40) # выводить больше столбцов
pd.set_option('display.max_rows', 40)   # выводить больше строк
plt.style.use('default')    # корректный вывод графиков в темной теме 

# fig.show(renderer='notebook') #включение анимации

r = requests.get("https://lms.skillfactory.ru/assets/courseware/v1/6559ab1e1d17acac79bec5dc8052261b/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/diabetes_data.zip")
with r, zipfile.ZipFile(io.BytesIO(r.content)) as archive:
    archive.extractall('Data')


# https://lms.skillfactory.ru/assets/courseware/v1/6559ab1e1d17acac79bec5dc8052261b/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/diabetes_data.zip

In [386]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


In [387]:
fig = px.histogram(diabetes, x="Pregnancies",  marginal="box", nbins=30, width=1000 , height= 500, color='Outcome')
fig.show()

In [388]:
diabetes_dd = diabetes.drop_duplicates()
diabetes_dd.shape

(768, 10)

In [389]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes_dd.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_dd[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_dd[col].nunique() / diabetes_dd[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.99:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.99:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
print(low_information_cols)

Gender: 100.0% одинаковых значений
['Gender']


In [390]:
diabetes_dd = diabetes_dd.drop(low_information_cols, axis=1)
diabetes_dd.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98,58,33,190,34.0,0.430,43,0
1,2,112,75,32,0,35.7,0.148,21,0
2,2,108,64,0,0,30.8,0.158,21,0
3,8,107,80,0,0,24.6,0.856,34,0
4,7,136,90,0,0,29.9,0.210,50,0


In [391]:
# Задание 8.3
# Попробуйте найти пропуски в данных с помощью метода insull().
# Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI
# обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!
# Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.
# Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.
diabetes_dd['Outcome']=diabetes_dd['Outcome'].astype('category')
diabetes_dd.info()
# display(df.colums())
for col in diabetes_dd.columns:
    if col != "Outcome":
        diabetes_dd[col] = diabetes_dd[col].replace({0:np.nan})
display(diabetes_dd.head())

round(diabetes_dd['Insulin'].isna().value_counts(normalize=True)[True], 2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Pregnancies               768 non-null    int64   
 1   Glucose                   768 non-null    int64   
 2   BloodPressure             768 non-null    int64   
 3   SkinThickness             768 non-null    int64   
 4   Insulin                   768 non-null    int64   
 5   BMI                       768 non-null    float64 
 6   DiabetesPedigreeFunction  768 non-null    float64 
 7   Age                       768 non-null    int64   
 8   Outcome                   768 non-null    category
dtypes: category(1), float64(2), int64(6)
memory usage: 54.9 KB


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,98.0,58.0,33.0,190.0,34.0,0.430,43,0
1,2.0,112.0,75.0,32.0,NaN,35.7,0.148,21,0
2,2.0,108.0,64.0,NaN,NaN,30.8,0.158,21,0
3,8.0,107.0,80.0,NaN,NaN,24.6,0.856,34,0
4,7.0,136.0,90.0,NaN,NaN,29.9,0.210,50,0


0.49

In [392]:
# Задание 8.4
# Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?
display(diabetes_dd.shape)
n = diabetes_dd.shape[0] #число строк в таблице
thresh_col = n*0.7
diabetes_dd = diabetes_dd.dropna(how='any', thresh=thresh_col, axis=1)
display(diabetes_dd)
display(diabetes_dd.shape)


(768, 9)

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,98.0,58.0,33.0,34.0,0.430,43,0
1,2.0,112.0,75.0,32.0,35.7,0.148,21,0
2,2.0,108.0,64.0,NaN,30.8,0.158,21,0
3,8.0,107.0,80.0,NaN,24.6,0.856,34,0
4,7.0,136.0,90.0,NaN,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...
763,5.0,139.0,64.0,35.0,28.6,0.411,26,0
764,1.0,96.0,122.0,NaN,22.4,0.207,27,0
765,10.0,101.0,86.0,37.0,45.6,1.136,38,1
766,NaN,141.0,NaN,NaN,42.4,0.205,29,1


(768, 8)

In [393]:
# Задание 8.5
# Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

display(diabetes_dd.shape)

m = diabetes_dd.shape[1]
tresh_line = m - 2 # убираем те записи где больее 2х Nan
print(f'triesh_line = {tresh_line}')
diabetes_dd = diabetes_dd.dropna(how='any', thresh=tresh_line, axis=0)

display(diabetes_dd.info())

display(diabetes_dd.shape)

# values = {
#     'one': df['one'].mean(),
#     'two': df['two'].mean(),
#     'three': df['three'].mean(),
#     'four': df['four'].mode()[0]
# }


(768, 8)

triesh_line = 6
<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Pregnancies               651 non-null    float64 
 1   Glucose                   747 non-null    float64 
 2   BloodPressure             733 non-null    float64 
 3   SkinThickness             541 non-null    float64 
 4   BMI                       748 non-null    float64 
 5   DiabetesPedigreeFunction  752 non-null    float64 
 6   Age                       752 non-null    int64   
 7   Outcome                   752 non-null    category
dtypes: category(1), float64(6), int64(1)
memory usage: 47.9 KB


None

(752, 8)

In [374]:
# Задание 8.6
# # В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.
print(round(diabetes_dd['SkinThickness'].mean(),1))
cols = list(diabetes_dd.columns)
cols.remove('Outcome')
print(cols)
for col in diabetes_dd.columns:
    if col != 'Outcome':
        diabetes_dd[col] = diabetes_dd[col].fillna(diabetes_dd[col].median())

display(round(diabetes_dd['SkinThickness'].mean(),1))

29.2
['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'BMI', 'DiabetesPedigreeFunction', 'Age']


29.1

In [375]:
# Задание 8.7

# Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?
def outliers_iqr_mod(data, feature, left =1.5, right= 1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(diabetes_dd, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 81
Результирующее число записей: 671
